In [3]:
import pandas as pd
import numpy as np
from unidecode import unidecode


In [4]:
df = pd.read_csv(r"C:\Users\luna love\Downloads\luxury-housing-bengaluru\Luxury_Housing_Bangalore.csv")
df.head()


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4bhk,12.750846039118798,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,16.292151871065954,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,NaN
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4bhk,10.517724412961911,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,9.396367494232896,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,15.345392444511946,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.


In [ ]:
# Clean text fields
def normalize_text(s):
    if pd.isna(s):
        return s
    s = unidecode(str(s)).strip()
    return " ".join(w.capitalize() for w in s.split())

df["Configuration"] = df["Configuration"].astype(str).str.upper().str.replace(" ", "")
df.head()


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4BHK,12.750846039118798,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292151871065954,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,NaN
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724412961911,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,9.396367494232896,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4BHK,15.345392444511946,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.


In [6]:
# Clean ticket price
def clean_ticket_price(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", "").replace("₹", "").replace("Cr", "").strip()
    try:
        return float(s)
    except:
        return np.nan

df["Ticket_Price_Cr"] = df["Ticket_Price_Cr"].apply(clean_ticket_price)


In [8]:
# Feature engineering
# Price per sqft
df["price_per_sqft"] = (df["Ticket_Price_Cr"] * 10_000_000) / df["Unit_Size_Sqft"]

# Quarter fields
df["Purchase_Quarter"] = pd.to_datetime(df["Purchase_Quarter"], errors="coerce")
df["quarter_label"] = df["Purchase_Quarter"].dt.to_period("Q").astype(str)
df["quarter_number"] = df["Purchase_Quarter"].dt.quarter

# NRI flag
df["nri_buyer_flag"] = df["NRI_Buyer"].astype(str).str.lower().map({"yes": 1, "no": 0})


In [10]:
# Save cleaned data
df.to_csv("cleaned_luxury_housing.csv", index=False)
print("Saved cleaned dataset as cleaned_luxury_housing.csv")


Saved cleaned dataset as cleaned_luxury_housing.csv


In [13]:
import pandas as pd
from sqlalchemy import create_engine

# Connection details
user = "root"
password = "Ponnusamy%402730"  # encode '@' as %40
host = "localhost"
port = 3306
db = "realestate"
table = "luxury_housing_sales"

# Build connection string
url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}"

# Create engine
engine = create_engine(url)

# Load cleaned CSV
df = pd.read_csv("cleaned_luxury_housing.csv")
df.to_sql(table, engine, if_exists="replace", index=False)

print(f"✅ Loaded {len(df)} rows into table {table}")


✅ Loaded 101000 rows into table luxury_housing_sales


In [15]:
import pandas as pd
from sqlalchemy import create_engine

# Connection details
user = "root"
password = "Ponnusamy%402730"   # '@' must be encoded as %40
host = "localhost"
port = 3306
db = "realestate"
table = "luxury_housing_sales_cleaned"

# Build connection string
url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}"
engine = create_engine(url)

# Load cleaned CSV
df = pd.read_csv("cleaned_luxury_housing.csv")

# ✅ Fix NRI_Buyer column: convert yes/no → 1/0
if "NRI_Buyer" in df.columns:
    df["NRI_Buyer"] = df["NRI_Buyer"].astype(str).str.lower().map({"yes": 1, "no": 0}).fillna(0).astype(int)

# Insert into MySQL
df.to_sql(table, engine, if_exists="append", index=False)

print(f"✅ Inserted {len(df)} rows into {table}")


✅ Inserted 101000 rows into luxury_housing_sales_cleaned
